# Backpropagation Though Time(BPTT)

RNN의 기본 계산 수식

<img src="./RNN_Tutorial_Part3_images/1.png" width=400 />

Loss는 Cross entropy로 정의

<img src="./RNN_Tutorial_Part3_images/2.png" width=400 />

여기서

<img src="./RNN_Tutorial_Part3_images/3.png" width=50 />

는 시간 스텝 t에서 실제 단어이고

<img src="./RNN_Tutorial_Part3_images/4.png" width=50 />

는 우리의 예측값이다.

보통 전체 시퀀스(문장)을 하나의 학습 데이터로 생각하고, 총 에러는 매 시간 스텝마다의 에러의 총 합으로 취한다.

<img src="./RNN_Tutorial_Part3_images/5.png" width=600 />

우리의 원래 목표는 파라미터 U, V, W에 대한 에러의 gradient를 계산해서 SGD를 이용해 좋은 파라미터를 찾는 것이다. <br />
에러들을 더하듯이 매 시간 스텝의 gradient도 하나의 학습데이터에 대해 모두 더해준다. 

<img src="./RNN_Tutorial_Part3_images/6.png" width=400 />

이 gradient를 계산하기 위해서는 미분의 Chain rule이 사용된다. <br />
에러에서부터 거꾸로 된 방향으로 계산하는 것이 결국 Backpropagation 알고리즘이 된다. 

예시로

<img src="./RNN_Tutorial_Part3_images/7.png" width=50 />

를 사용하였다.

<img src="./RNN_Tutorial_Part3_images/8.png" width=300 />

위 식에서

<img src="./RNN_Tutorial_Part3_images/9.png" width=200 />

이고

<img src="./RNN_Tutorial_Part3_images/10.png" width=50 />

는 두 벡터의 외적이다. 

여기서 핵심 포인트는 

<img src="./RNN_Tutorial_Part3_images/11.png" width=50 />

가 현재 시간 스텝의

<img src="./RNN_Tutorial_Part3_images/12.png" width=200 />

에만 의존한다는 점이다.

이 세 값을 갖고 있다면 V에 대한 gradient를 계산하는 것은 단순한 행렬곱이 된다.

그러나,

<img src="./RNN_Tutorial_Part3_images/13.png" width=100 />

에 대해서는(U에 대해서도) 상황이 조금 다르다. 이를 위해 Chain rule을 이용하여 전개해 보면

여기서 s(t) =tanh(U(x(t)+W(s(t-1))))는 s(2)에 의존하고, s(2)는 W와 s(1)에 의존해서 Chain rule이 계속 이어진다. <br />
따라서 W에 대한 미분을 하기 위해서는 s(2)를 단순히 상수로 취급하면 안된다. <br />
다시 Chain rule을 적용하면 아래 식을 얻을 수 있다.

<img src="./RNN_Tutorial_Part3_images/14.png" width=500 />

각 시간 스텝이 gradient에 기여하는 것을 전부 더해준다. 즉 W는 우리가 현재 처리중인 출력부분까지의 모든 시간 스텝에서 <br />
사용되기 때문에 t=3 부터 t=0 까지 gradient들을 전부 backpropagate 해 주어야 한다.

<img src="./RNN_Tutorial_Part3_images/16.png" width=600 />

이 과정은 DNN에 사용하는 원래의 backpropagation 알고리즘과 똑같은 것을 알 수 있다. <br />
중요한 차이점은 매 시간 스텝마다 W에 대한 gradient를 더해준다는 점이다. <br />
기존의 신경망 구조에서는 layer별로 파라미터를 공유하지 않기 때문에 계산 결과들을 서로 더해 줄 필요가 없다. <br />

# Vanishing Gradient 문제

Gradient 식을 다시 한 번 살펴보자

<img src="./RNN_Tutorial_Part3_images/17.png" width=500 />

여기서 주목해야 하는 것은

<img src="./RNN_Tutorial_Part3_images/18.png" width=50 />

도 Chain rule을 내포하고 있다는 점이다. 즉,

<img src="./RNN_Tutorial_Part3_images/19.png" width=200 />

이다. 또 하나는 벡터를 벡터로 미분하고 있기 때문에 결과는 자코비안 행렬이 나오게 된다.<br />
위의 Gradient 식을 다시 써보면

<img src="./RNN_Tutorial_Part3_images/20.png" width=500 />

위 자코비안 행렬의 2-norm(행렬의 절대값)값의 최대값이 1이라는 것이 증명되었다. <br />
현재 사용하고 있는 activation 함수인 tanh(또는 sigmoid)는 모든 값을 -1 부터 1 까지로 매핑시켜주고,<br />
미분값은 최대 1로(sigmoid의 경우 1/4) 정해지기 때문이라고 할 수 있다. 

tanh함수와 sigmoid함수는 양쪽 끝에서 미분값이 0으로 수렴하는 것을 볼 수 있다. <br />
이 현상이 발생할 때 그 뉴런이 포화되었다고 말하는데, 이런 뉴런들은 gradient가 거의 0이기 때문에 곱해지는 이전 layer의 <br />
gradient들도 0으로 수렴하게 만든다. 따라서 행렬에 작은 값들이 들어있고 여러번 행렬곱이 이루어 지면 gradient는 지수 함수로 감소하고 <br />
시간 스텝 몇 번만 지나도 사라져버린다. <br />
시퀀스에서 여러 시간 스텝이 떨어진 곳에서는 gradient가 전달되지 못하고, 먼 과거의 상태(state)는 현재 스텝의 학습에
아무 도움이 되지 못하게 된다.

## 해결책

### cf) Exploding Gradient

Gradient 계산을 보면, 자코비안 행렬 안의 값들이 크다면 activation 함수와 네트워크 파라미터 값에 따라 <br />
gradient가 사라지게 되는 것이 아니라 지수 함수로 증가하는 경우도 충분히 상상해 볼 수 있다. <br />
이런 문제의 경우 NaN(not a number)이 될 것이고 프로그램이 죽게됨으로 쉽게 알아차릴 수 있다. <br />
따라서 미리 정해둔 적당한 값ㅇ로 잘라버리는 방법을 이용하면 매우 쉽고 효율적으로 이 문제를 해결 할 수 있다.

- W 행렬을 적당히 좋은 값으로 잘 초기화 한다.
- Regulation을 잘 정한다.
- tanh나 sigmoid를 activation 함수로 사용하지말고 ReLU를 사용한다. 